# NVIDIA NIM Config

In [56]:
import os
from langchain_nvidia_ai_endpoints import ChatNVIDIA
import time
from docling_core.types.doc import ImageRefMode, PictureItem, TableItem
from docling.datamodel.base_models import FigureElement, InputFormat, Table
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption


# 从文件中读取 API key
def read_api_keys(file_path: str) -> dict:
    """
    Reads the API keys from the specified file.

    Args:
        file_path (str): The path to the file containing the API keys.

    Returns:
        dict: A dictionary with the API keys.
    """
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
            api_keys = {
                "NVIDIA_API_KEY": lines[0].strip().strip('["').strip('"]'),
                "TTS_APIKEY": lines[1].strip().strip("',").strip("']")
            }
        return api_keys
    except FileNotFoundError:
        raise Exception(f"File not found: {file_path}")
    except Exception as e:
        raise Exception(f"An error occurred while reading the API keys: {str(e)}")


# Example usage
api_keys = read_api_keys('./nv_api_key.txt')
NV_API_KEY = api_keys["NVIDIA_API_KEY"]
TTS_API_KEY = api_keys["TTS_APIKEY"]

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"  # conda environments have to set this
os.environ["NVIDIA_API_KEY"] = NV_API_KEY
ChatNVIDIA.get_available_models()

[Model(id='nvidia/llama-3.1-nemotron-70b-reward', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=None, supports_tools=False, supports_structured_output=False, base_model=None),
 Model(id='mistralai/mixtral-8x22b-instruct-v0.1', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=['ai-mixtral-8x22b-instruct'], supports_tools=False, supports_structured_output=False, base_model=None),
 Model(id='nvidia/llama-3.1-nemotron-51b-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=None, supports_tools=False, supports_structured_output=False, base_model=None),
 Model(id='nv-mistralai/mistral-nemo-12b-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=None, supports_tools=True, supports_structured_output=True, base_model=None),
 Model(id='databricks/dbrx-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=['ai-dbrx-instruct'], supports_tools=False, supports_structured_output=False, base_model=None),
 Mo

# Convert PDF to Markdown

In [57]:
import logging

_log = logging.getLogger(__name__)
IMAGE_RESOLUTION_SCALE = 2.0
markdown_file_path = None
output_dir = None


def save_page_images(conv_res, output_dir, doc_filename):
    """Save images of each page."""
    for page_no, page in conv_res.document.pages.items():
        page_no = page.page_no
        page_image_filename = output_dir / f"{doc_filename}-{page_no}.png"
        with page_image_filename.open("wb") as fp:
            page.image.pil_image.save(fp, format="PNG")


def save_figures_and_tables(conv_res, output_dir, doc_filename):
    """Save images of figures and tables."""
    table_counter = 0
    picture_counter = 0
    for element, _level in conv_res.document.iterate_items():
        if isinstance(element, TableItem):
            table_counter += 1
            element_image_filename = (
                    output_dir / f"{doc_filename}-table-{table_counter}.png"
            )
            with element_image_filename.open("wb") as fp:
                element.get_image(conv_res.document).save(fp, "PNG")

        if isinstance(element, PictureItem):
            picture_counter += 1
            element_image_filename = (
                    output_dir / f"{doc_filename}-picture-{picture_counter}.png"
            )
            with element_image_filename.open("wb") as fp:
                element.get_image(conv_res.document).save(fp, "PNG")


def save_markdown_with_embedded_pictures(conv_res, output_dir, doc_filename):
    """Save markdown with embedded pictures."""
    md_filename = output_dir / f"{doc_filename}-with-images.md"
    conv_res.document.save_as_markdown(md_filename, image_mode=ImageRefMode.EMBEDDED)


def save_markdown_with_externally_referenced_pictures(conv_res, output_dir, doc_filename):
    """Save markdown with externally referenced pictures."""
    md_filename = output_dir / f"{doc_filename}-with-image-refs.md"
    conv_res.document.save_as_markdown(md_filename, image_mode=ImageRefMode.REFERENCED)


def convert_pdf_to_markdown(input_doc_path: str):
    global markdown_file_path
    global output_dir
    logging.basicConfig(level=logging.INFO)

    input_doc_path = Path(f"{input_doc_path}")
    output_dir = Path(f"{input_doc_path.stem}-{datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}")
    markdown_storage_dir = Path(f"data/markdown/{output_dir}")
    markdown_file_path = markdown_storage_dir / f"{input_doc_path.stem}-with-image-refs.md"

    pipeline_options = PdfPipelineOptions()
    pipeline_options.images_scale = IMAGE_RESOLUTION_SCALE
    pipeline_options.generate_page_images = True
    pipeline_options.generate_picture_images = True

    doc_converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
        }
    )

    start_time = time.time()

    conv_res = doc_converter.convert(input_doc_path)

    markdown_storage_dir.mkdir(parents=True, exist_ok=True)
    doc_filename = conv_res.input.file.stem

    save_page_images(conv_res, markdown_storage_dir, doc_filename)
    save_figures_and_tables(conv_res, markdown_storage_dir, doc_filename)
    save_markdown_with_embedded_pictures(conv_res, markdown_storage_dir, doc_filename)
    save_markdown_with_externally_referenced_pictures(conv_res, markdown_storage_dir, doc_filename)

    end_time = time.time() - start_time

    _log.info(f"Document converted and figures exported in {end_time:.2f} seconds.")
    print(f"Markdown file saved at {markdown_file_path}")

# FAISS Config

In [58]:
from langchain_community.vectorstores import FAISS
from typing import Tuple, List
from pathlib import Path
from typing import List, Dict, Any
from langchain.text_splitter import CharacterTextSplitter
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings

faiss_store_path_PDF = Path("data/faiss_store/PDF")
store_PDF = None
retriever_PDF = None

faiss_store_path_chart = Path("data/faiss_store/chart")
store_chart = None
retriever_chart = None


def read_markdown_content(file_path: Path) -> Tuple[List[str], List[str]]:
    """
    Read and filter markdown content from a file.

    Args:
        file_path: Path to the markdown file

    Returns:
        Tuple containing:
            - List[str]: Filtered content lines
            - List[str]: Corresponding source paths for each line
    """
    data: List[str] = []
    sources: List[str] = []

    try:
        with open(file_path, encoding="utf-8") as f:
            for line in f:
                line = line.strip()
                if line:  # Check for non-empty lines
                    data.append(line)
                    sources.append(str(file_path))

        print(f"Read {len(data)} lines from {file_path}")
        return data, sources

    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return [], []
    except Exception as e:
        print(f"Error reading file: {e}")
        return [], []


# # Usage
# data, sources = read_markdown_content(markdown_file_path)
# documents = data  # Since we already filtered empty lines during reading

def create_faiss_store(
        documents: List[str],
        sources: List[str],
        store_path: Path,
        chunk_size: int = 400,
        model: str = "NV-Embed-QA"
) -> FAISS:
    """
    Create and save a FAISS vector store from documents.

    Args:
        documents: List of document texts
        sources: List of source paths
        store_path: Path to save the FAISS store
        chunk_size: Size of text chunks for splitting
        model: NVIDIA embedding model name

    Returns:
        FAISS: The created FAISS store
    """
    embedder = NVIDIAEmbeddings(model=model)
    text_splitter = CharacterTextSplitter(chunk_size=chunk_size, separator=" ")

    docs: List[str] = []
    metadatas: List[Dict[str, Any]] = []

    for i, doc in enumerate(documents):
        splits = text_splitter.split_text(doc)
        docs.extend(splits)
        metadatas.extend([{"source": sources[i]} for _ in splits])

    store = FAISS.from_texts(docs, embedder, metadatas=metadatas)
    store.save_local(store_path)
    return store

# # Example usage
# documents = ["Document 1 text", "Document 2 text"]
# sources = ["source1.md", "source2.md"]
# store_path = Path("data/faiss_store/example")
# 
# store_PDF = create_faiss_store(documents, sources, store_path)



# Langchain Config

In [59]:

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_nvidia_ai_endpoints import ChatNVIDIA

llm = ChatNVIDIA(model="meta/llama3-70b-instruct")

article_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert IELTS English teacher. Based on the provided context, help the student understand the article. "
            "Explain difficult vocabulary, summarize key points, and provide examples where necessary. "
            "Use clear and concise language. "
            "Answer solely based on the following context:\n<Documents>\n{context}\n</Documents>",
        ),
        (
            "user",
            "请帮助我理解文章的主要内容和难词。",
        ),
    ]
)

podcast_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert IELTS English teacher and podcast scriptwriter. Based on the provided context, create a podcast script for the student. "
            "The script should be engaging, colloquial, and suitable for an IELTS proficiency level. "
            "Focus on summarizing the key points of the article, explaining difficult vocabulary, and providing examples where necessary. "
            "Do not include any timestamps, segment labels, or additional notes. "
            "The script should be approximately 4 minutes long when spoken. "
            "Start the script with '<podcast>' and end it with '</podcast>'. "
            "Answer solely based on the following context:\n<Documents>\n{context}\n</Documents>"
        ),
        (
            "user",
            "请帮助我生成一个符合雅思能力水平的播客节目文稿，时长约为4分钟。",
        ),
    ]
)
chart_analysis_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a data analysis expert and IELTS academic writing instructor. Based on the provided chart data, generate a model essay for IELTS Writing Task 1. "
            "The essay should strictly follow academic writing standards and demonstrate professional chart interpretation skills. "
            "Required elements:\n"
            "1. Precise identification of chart type (line/bar/pie etc.) and main trends\n"
            "2. Logical grouping of data features with comparative analysis\n"
            "3. Accurate use of statistical terminology and percentage expressions\n"
            "4. Grammatical diversity in data description (passive/active voice conversion)\n"
            "5. Clear overview paragraph highlighting most significant information\n"
            "6. Word count between 150-200 words\n"
            "Format requirements:\n"
            "- Use <essay> tags to wrap content\n"
            "- Avoid markdown formatting\n"
            "- Prohibit speculative statements beyond given data\n"
            "Reference only the following data:\n<Chart>\n{context}\n</Chart>"
        ),
        (
            "user",
            "请根据提供的图表数据，生成一篇符合雅思学术写作标准的图表分析范文。",
        ),
    ]
)

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.vectorstores import FAISS
from langchain_nvidia_ai_endpoints import ChatNVIDIA


def create_chains(model: str = "meta/llama3-70b-instruct"):
    """
    Create article and podcast processing chains.

    Args:

        model: LLM model name

    Returns:
        tuple: (article_chain, podcast_chain)
    """

    llm = ChatNVIDIA(model=model)
    global retriever_PDF
    global retriever_chart

    if retriever_PDF is None:
        article_chain = None
        podcast_chain = None
    else:
        article_chain = (
                {"context": retriever_PDF, "question": RunnablePassthrough()}
                | article_prompt
                | llm
                | StrOutputParser()
        )

        podcast_chain = (
                {"context": retriever_PDF, "question": RunnablePassthrough()}
                | podcast_prompt
                | llm
                | StrOutputParser()
        )
    if retriever_chart is None:
        chart_chain = None
    else:
        chart_chain = (
                {"context": retriever_chart, "question": RunnablePassthrough()}
                | chart_analysis_prompt
                | llm
                | StrOutputParser()
        )

    return article_chain, podcast_chain, chart_chain


# Podcast Config

In [60]:
# Podcast Logic
import soundfile as sf
import numpy as np
from pathlib import Path

voice_list = [
    "English-US.Female-1",
    "English-US.Male-1",
    "English-US.Female-Neutral",
    "English-US.Male-Neutral",
    "English-US.Female-Angry",
    "English-US.Male-Angry",
    "English-US.Female-Calm",
    "English-US.Male-Calm",
    "English-US.Female-Fearful",
    "English-US.Female-Happy",
    "English-US.Male-Happy",
    "English-US.Female-Sad"
]


def split_podcast_text(podcast_text: str, max_length: int = 600) -> list[str]:
    """
    将包含 <podcast> 标签的文本分割为段落列表，并尽量合并段落，确保每个段落字符数不超过 max_length。

    Args:
        podcast_text (str): 生成的播客文稿文本（包含 <podcast> 标签）
        max_length (int): 每个段落的字符数上限，默认为 1000

    Returns:
        list[str]: 合并后的段落列表，例如 ["para1", "para2", ...]
    """
    # 1. 去除 <podcast> 标签并提取正文内容
    content = podcast_text.strip()  # 去除首尾空格
    content = content.replace("<podcast>", "").replace("</podcast>", "").strip()

    # 2. 按换行符分割段落，并过滤空段落
    paragraphs = [p.strip() for p in content.split("\n") if p.strip()]

    # 3. 合并段落，确保每个段落字符数不超过 max_length
    merged_paragraphs = []
    current_paragraph = ""

    for paragraph in paragraphs:
        # 如果当前段落加上新段落的长度不超过 max_length，则合并
        if len(current_paragraph) + len(paragraph) + 1 <= max_length:
            current_paragraph += (" " + paragraph) if current_paragraph else paragraph
        else:
            # 如果当前段落不为空，则添加到结果列表
            if current_paragraph:
                merged_paragraphs.append(current_paragraph)
            # 开始新的段落
            current_paragraph = paragraph

    # 添加最后一个段落
    if current_paragraph:
        merged_paragraphs.append(current_paragraph)

    return merged_paragraphs


def synthesize_speech(text: str, voice: str, output: str) -> None:
    import time
    import wave
    import json
    import riva.client
    from pathlib import Path

    # Authentication & service creation
    auth = riva.client.Auth(None, True, 'grpc.nvcf.nvidia.com:443',
                            [['function-id', '0149dedb-2be8-4195-b9a0-e57e0e14f972'],
                             ['authorization',
                              f'{TTS_API_KEY}']])
    service = riva.client.SpeechSynthesisService(auth)

    # Prepare audio file for writing
    out_path = Path(output).expanduser()
    out_f = wave.open(str(out_path), 'wb')
    out_f.setnchannels(1)
    out_f.setsampwidth(2)
    out_f.setframerate(44100)

    try:
        print("Generating audio for request...")
        start = time.time()
        response = service.synthesize(
            text, voice, "en-US", sample_rate_hz=44100,
            audio_prompt_file=None, quality=20, custom_dictionary={}
        )
        print(f"Time spent: {(time.time() - start):.3f}s")
        out_f.writeframesraw(response.audio)
    finally:
        out_f.close()


def combine_audio_files(input_dir: str, output_file: str) -> None:
    """
    Combine all WAV files in the specified directory into a single audio file.

    Args:
        input_dir (str): The directory containing the WAV files to combine.
        output_file (str): The path to the output combined audio file.
    """
    input_path = Path(input_dir)
    wav_files = sorted(input_path.glob("podcast_audio-*.wav"))

    combined_audio = []
    for wav_file in wav_files:
        data, samplerate = sf.read(wav_file)
        combined_audio.extend(data)

    combined_audio = np.array(combined_audio)
    sf.write(output_file, combined_audio, samplerate)

# # Example usage
# combine_audio_files("./data/podcast", "./data/podcast/combined_podcast.wav")

# Gradio Config

In [61]:
import os
import shutil
import base64
import datetime
from pathlib import Path
import gradio as gr
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from typing import List

TARGET_DIR_PDF = "data/pdf"
pdf_file_path = ""
podcast_text_plain = ""
podcast_text_split = []

TARGET_DIR_CHART = "data/charts"
chart_file_path = ""
markdown_file_path_chart = ""
cur_chart_dir = ""


def save_pdf(file, progress=gr.Progress()):
    progress(0, desc="Starting task...")
    if file is None:
        return "No file uploaded", None, None

    # 确保目标目录存在
    if not os.path.exists(TARGET_DIR_PDF):
        os.makedirs(TARGET_DIR_PDF)

    try:
        # 获取文件名
        file_name = os.path.basename(file.name)
        # 目标文件路径
        target_path = os.path.join(TARGET_DIR_PDF, file_name)
        global pdf_file_path
        pdf_file_path = target_path
        print(pdf_file_path)
        # 复制文件到目标目录
        shutil.copy(file.name, target_path)
        progress(0.20, desc="File uploaded successfully!")
        convert_pdf_to_markdown(f"{pdf_file_path}")
        progress(0.40, desc="PDF converted to markdown!")
        data, sources = read_markdown_content(markdown_file_path)
        global store_PDF
        store_PDF = create_faiss_store(
            documents=data,
            sources=sources,
            store_path=faiss_store_path_PDF
        )
        progress(0.75, desc="Faiss Create successfully!")
        global retriever_PDF
        retriever_PDF = store_PDF.as_retriever()
        progress(1, desc="Embedding and Faiss store created!")
        alert_message = "Embedding and Faiss completed! You can start asking questions now."
        return (gr.update(visible=True, value=alert_message),
                gr.update(value=pdf_file_path), gr.update(value=pdf_file_path))
    except Exception as e:
        return (f"Error: {str(e)}", gr.update(visible=True, value=f"Error: {str(e)}"),
                None, None)


def image2b64(image_file):
    with open(image_file, "rb") as f:
        image_b64 = base64.b64encode(f.read()).decode()
        return image_b64


def save_chart(file_path: str, progress=gr.Progress()):
    progress(0, desc="Starting task...")
    if not file_path:
        return "No file uploaded", None, None

    # Ensure the target directory exists
    if not os.path.exists(TARGET_DIR_CHART):
        os.makedirs(TARGET_DIR_CHART)

    try:
        # Get the file name
        file_name = os.path.basename(file_path)
        # Target file path
        global cur_chart_dir
        cur_chart_dir = os.path.join(TARGET_DIR_CHART, f"{datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}")
        os.makedirs(cur_chart_dir, exist_ok=True)

        gr.update(visible=True, value="waiting for chart description...")

        global chart_file_path
        chart_file_path = os.path.join(cur_chart_dir, file_name)
        print("chart_file_path:")
        print(chart_file_path)

        # Copy the file to the target directory
        shutil.copy(file_path, chart_file_path)
        progress(0.20, desc="File uploaded successfully!")

        image_b64 = image2b64(chart_file_path)
        chart_reading = ChatNVIDIA(model="microsoft/phi-3-vision-128k-instruct")
        result = chart_reading.invoke(
            f'Generate underlying data table of the figure below, : <img src="data:image/png;base64,{image_b64}" />')
        print(result)

        # Save result text to markdown file
        global markdown_file_path_chart
        markdown_file_path_chart = Path(f"{cur_chart_dir}/chart.md")
        print("markdown_file_path_chart")
        print(markdown_file_path_chart)
        with open(markdown_file_path_chart, "w") as f:
            f.write(result.content)

        progress(0.40, desc="Chart converted to markdown!")
        print("Chart converted to markdown!" + str(markdown_file_path_chart))

        # Read markdown content (assuming a function read_markdown_content exists)
        data, sources = read_markdown_content(markdown_file_path_chart)

        # Create FAISS store (assuming a function create_faiss_store exists)
        global store_chart
        store_chart = create_faiss_store(
            documents=data,
            sources=sources,
            store_path=faiss_store_path_chart
        )
        progress(0.75, desc="FAISS store created successfully!")

        global retriever_chart
        retriever_chart = store_chart.as_retriever()
        progress(1, desc="Embedding and FAISS store created!")

        return gr.update(visible=True, value=result.content, lines=8)
    except Exception as e:
        return gr.update(visible=True, value="Error: " + str(e))


def article_agent(message: str, history: List[List[str]]) -> str:
    """
    Process user messages and generate responses using the article chain.

    Args:
        message: User's input message
        history: Chat history as list of [user_message, bot_message] pairs

    Returns:
        str: Generated response
    """
    try:
        article_chain, _, _ = create_chains()
        response = article_chain.invoke(message)
        # response = "This is a response, you need to implement the response generation logic."
        return response
    except Exception as e:
        return f"Error processing request: {str(e)}"


def respond(message, chat_history):
    bot_message = article_agent(message, chat_history)
    chat_history.append([message, bot_message])
    return "", chat_history


def podcast_agent(message: str, progress=gr.Progress()) -> str:
    try:
        _, podcast_chain, _ = create_chains()
        progress(0.20, desc="Podcast script chain builds successfully!")
        global podcast_text_plain
        if message is None:
            podcast_text_plain = podcast_chain.invoke(
                "Generate an English podcast script for an IELTS proficiency level.")
            progress(0.60, desc="Podcast script generated successfully!")
        else:
            podcast_text_plain = podcast_chain.invoke(message)

            podcast_text_plain = podcast_text_plain.strip()  # 去除首尾空格
            podcast_text_plain = podcast_text_plain.replace("<podcast>", "").replace("</podcast>", "").strip()
            podcast_storage_dir = Path(f"data/podcast/{output_dir}")
            progress(0.80, desc="Podcast script generated successfully!")
            podcast_storage_dir.mkdir(parents=True, exist_ok=True)
            with open(f"./data/podcast/{output_dir}/podcast_script_text.txt", "w") as f:
                f.write(podcast_text_plain)
            progress(1, desc="Podcast script saved successfully!")

        return podcast_text_plain
    except Exception as e:
        return f"Error processing request: {str(e)}"


def podcast_audio_generate(progress=gr.Progress()):
    global podcast_text_split
    podcast_text_list = split_podcast_text(podcast_text_plain)

    for i, text in enumerate(podcast_text_list):
        progress(0.10 + i * 0.80 / len(podcast_text_list), desc=f"Generating audio for paragraph {i + 1}...")
        synthesize_speech(text=text, voice="English-US.Male-Neutral",
                          output=f"./data/podcast/{output_dir}/podcast_audio-{i}.wav")

    progress(0.8, desc="Audio files generated successfully!")

    combine_audio_files(f"./data/podcast/{output_dir}", f"./data/podcast/{output_dir}/combined_podcast.wav")
    progress(1, desc="Audio files combined successfully!")
    return gr.update(visible=True, value=f"./data/podcast/{output_dir}/combined_podcast.wav")


def chart_agent(message: str, progress=gr.Progress()):
    _, _, chart_chain = create_chains()
    if message is None:
        chart_text_plain = chart_chain.invoke(
            "Generate an English writing for an IELTS proficiency level.")
        progress(0.60, desc="Chart script generated successfully!")
    else:
        chart_text_plain = chart_chain.invoke(message)

        chart_text_plain = chart_text_plain.strip()  # 去除首尾空格
        chart_text_plain = chart_text_plain.replace("<essay>", "").replace("</essay>", "").strip()

        progress(0.80, desc="Chart script generated successfully!")

        with open(f"{cur_chart_dir}/chart_script_text.txt", "w") as f:
            f.write(chart_text_plain)
        progress(1, desc="Chart script saved successfully!")
    return chart_text_plain

In [ ]:
with gr.Blocks(theme="soft") as demo:
    with gr.Tab(label="Article Understanding"):
        gr.Markdown("# English Article Assistant")
        gr.Markdown("Upload an article and ask questions to understand it better.")
        with gr.Row():
       
                chatbot = gr.Chatbot(
                    label="Chat History",
                    height=500,
                    show_copy_button=True
                )
            

        with gr.Row(scale=1):
            with gr.Column(scale=3):
                msg = gr.Textbox(
                    label="Your Question",
                    placeholder="Ask about the article...",
                    lines=2
                )

            with gr.Column(scale=2):
                send = gr.Button("Send")
                clear = gr.Button("Clear")
                
            with gr.Column(scale=2):
                article_file = gr.File(
                    label="Upload PDF",
                    file_types=[".pdf"],
                    scale=1,
                    height=150
                )
                article_submit = gr.Button("Submit", scale=1)
        alert = gr.Textbox(visible=False, scale=2, lines=2, label="System message")

    with gr.Tab(label="Podcast Generate"):
        gr.Markdown("# English Podcast Generator")
        gr.Markdown("Generate a podcast script based on the uploaded article.")
        with gr.Row():
            with gr.Column(scale=3):
                podcast_text = gr.Textbox(
                    label="Podcast Script",
                    placeholder="Podcast script will appear here...",
                    lines=22,
                )
            with gr.Column(scale=2):
                podcast_file = gr.File(
                    label="Upload PDF",
                    file_types=[".pdf"],
                    scale=1,
                    height=150,

                )
                podcast_prompt_user = gr.Textbox(
                    label="podcast_prompt_user",
                    value="Generate an English podcast script for an IELTS proficiency level.",
                    lines=2)
                podcast_submit = gr.Button("Submit", scale=1)
                with gr.Row():
                    podcast_script_text_generate = gr.Button("Generate Text", scale=1)
                    podcast_script_audio_generate = gr.Button("Generate Audio", scale=1)
                podcast_audio = gr.Audio(visible=True)
    with gr.Tab(label="Chart&Table Description"):
        gr.Markdown("# Chart & Table Analyze")
        gr.Markdown("Upload a chart picture and generate an essay based on it.")
        with gr.Row():
            with gr.Column(scale=2):
                picture_table_file = gr.Image(label="Upload a Chart", scale=1, type="filepath", height=300)

                chart_description = gr.Textbox(label="Chart description", lines=4, visible=True,
                                               value="waiting for chart description...")
                with gr.Row():
                    picture_table_submit = gr.Button("Submit Chart", scale=1)
                    chart_essay_generate = gr.Button("Generate Essay", scale=1)
            with gr.Column(scale=3):
                chart_essay = gr.Textbox(label="Essay based on chart", lines=18)
                chart_user_input = gr.Textbox(label="chart_user_input",
                                              value="Generate an English writing for an IELTS proficiency level.",
                                              lines=2)

    # 绑定按钮点击事件
    article_submit.click(
        fn=save_pdf,  # 处理文件的函数
        inputs=article_file,  # 输入组件
        outputs=[alert, podcast_file, article_file],  # 输出组件
    )
    podcast_submit.click(
        fn=save_pdf,  # 处理文件的函数
        inputs=podcast_file,  # 输入组件
        outputs=[alert, podcast_file, article_file],  # 输出组件

    )

    send.click(
        respond,
        [msg, chatbot],
        [msg, chatbot],
        queue=False
    )

    clear.click(lambda: None, None, chatbot, queue=False)

    podcast_script_text_generate.click(
        fn=podcast_agent,
        inputs=podcast_prompt_user,
        outputs=podcast_text
    )

    podcast_script_audio_generate.click(
        fn=podcast_audio_generate,
        inputs=None,
        outputs=podcast_audio
    )

    picture_table_submit.click(
        fn=save_chart,
        inputs=picture_table_file,
        outputs=[chart_description]
    )

    chart_essay_generate.click(
        fn=chart_agent,
        inputs=chart_user_input,
        outputs=[chart_essay]
    )

demo.launch(demo, debug=True, share=False, server_port=15000)

C:\Users\LuckyE\miniconda3\envs\RAG-Proj\Lib\site-packages\gradio\components\chatbot.py:284: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:15000
